In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

/Users/tom/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
feature_list = ['similarity_score',
                'nn_out',
                'last_char',
                'avg_shared_words',
                'word_count_diff',
                'levenshtein',
                'shared_words_pcnt',
                'avg_shared_trigrams',
                'shared_bigram_pcnt',
                'shared_trigram_pcnt',
                'avg_shared_quadgrams',
                'shared_quadgram_pcnt',
                'tfidf_word_match_share']

In [3]:
nn_out = pd.read_csv('../roughfeaturedata/preds_for_logreg.csv')
train_features = pd.read_csv('../data/train_features.csv')

In [4]:
nn_out.columns = ['nn_out', 'id']
nn_out = nn_out.merge(train_features)
nn_out.head()

,nn_out,id,question1,question2,is_duplicate,last_char,avg_shared_words,word_count_diff,levenshtein,shared_words_pcnt,avg_shared_trigrams,shared_bigram_pcnt,shared_trigram_pcnt,avg_shared_quadgrams,shared_quadgram_pcnt
0,0.004252,28170,Is MPM & Partners in Monaco real?,All those who live in Monaco are fabulously we...,0,1,2,2,0.470588,0.250000,0,0.142857,0.000000,0,0.0
1,0.056201,29836,What's the fastest way to get a driver's licen...,How can you get a driver license in your country?,0,1,4,1,0.605505,0.380952,0,0.210526,0.000000,0,0.0
2,0.009561,208192,How do I integrate 1/ (1+x^4)?,How do I integrate |x|?,0,1,4,1,0.792453,0.727273,2,0.666667,0.571429,1,0.4
3,0.659983,96826,How is Odysseus a bad leader in the Odyssey?,"Why was Odysseus considered a good leader in ""...",1,1,4,1,0.718447,0.421053,0,0.117647,0.000000,0,0.0
4,0.173221,165584,Why was the ending of Aadhe adhoore (Indian Tv...,Why is Indian TV serials too dramatic?,0,1,2,9,0.475410,0.173913,0,0.000000,0.000000,0,0.0


In [5]:
train_sims = pd.read_csv('../roughfeaturedata/train_with_sim_and_ents_long.csv')
train_sims = train_sims[['id','similarity_score']]

In [6]:
merged = train_sims.merge(nn_out)
merged.head()

,id,similarity_score,nn_out,question1,question2,is_duplicate,last_char,avg_shared_words,word_count_diff,levenshtein,shared_words_pcnt,avg_shared_trigrams,shared_bigram_pcnt,shared_trigram_pcnt,avg_shared_quadgrams,shared_quadgram_pcnt
0,0,0.986684,0.419988,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,1,12,2,0.926829,0.923077,9,0.833333,0.818182,8,0.8
1,1,0.925000,0.043146,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,1,4,5,0.647482,0.380952,0,0.105263,0.000000,0,0.0
2,2,0.929068,0.078106,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,1,4,4,0.454545,0.333333,0,0.090909,0.000000,0,0.0
3,3,0.730742,0.002271,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,1,0,2,0.069565,0.000000,0,0.000000,0.000000,0,0.0
4,4,0.846616,0.003942,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,1,2,6,0.365217,0.200000,0,0.000000,0.000000,0,0.0


In [13]:
#add tfidf and LDA

tfidf = pd.read_csv('../data/tfidf_train_features.csv')
tfidf = tfidf[['id','tfidf_word_match_share']]
tfidf.columns = ['id', 'tfidf_word_match_share']

#topic_modelling = pd.read_csv('../data/topic_modelling_output.csv')


In [14]:
merged = merged.merge(tfidf)
#merged = merged.merge(topic_modelling)

In [15]:
#Drop nan
merged = merged.dropna()

In [16]:
y = merged['is_duplicate']
x = merged[feature_list]
x.head()

,similarity_score,nn_out,last_char,avg_shared_words,word_count_diff,levenshtein,shared_words_pcnt,avg_shared_trigrams,shared_bigram_pcnt,shared_trigram_pcnt,avg_shared_quadgrams,shared_quadgram_pcnt,tfidf_word_match_share
0,0.986684,0.419988,1,12,2,0.926829,0.923077,9,0.833333,0.818182,8,0.8,0.905803
1,0.925000,0.043146,1,4,5,0.647482,0.380952,0,0.105263,0.000000,0,0.0,0.305729
2,0.929068,0.078106,1,4,4,0.454545,0.333333,0,0.090909,0.000000,0,0.0,0.428620
3,0.730742,0.002271,1,0,2,0.069565,0.000000,0,0.000000,0.000000,0,0.0,0.000000
4,0.846616,0.003942,1,2,6,0.365217,0.200000,0,0.000000,0.000000,0,0.0,0.000000


In [17]:

logit_model=sm.Logit(y,x)
result=logit_model.fit()
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.248360
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:           is_duplicate   No. Observations:               305644
Model:                          Logit   Df Residuals:                   305631
Method:                           MLE   Df Model:                           12
Date:                Tue, 28 Nov 2017   Pseudo R-squ.:                  0.6232
Time:                        22:27:43   Log-Likelihood:                -75910.
converged:                       True   LL-Null:                   -2.0145e+05
                                        LLR p-value:                     0.000
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
similarity_score          -3.4569      0.053    -65.817      0.000      -3.560      

In [25]:
nn_out_test = pd.read_csv('../roughfeaturedata/test_preds_for_logreg.csv')
test_features = pd.read_csv('../data/test_features.csv')
tfidf_test_features = pd.read_csv('../data/tfidf_test_features.csv')
nn_out_test.columns = ['nn_out','test_id']
nn_out_test = nn_out_test.merge(test_features)
nn_out_test = nn_out_test.merge(tfidf_test_features)


In [26]:
test = pd.read_csv('../roughfeaturedata/test_with_sim_and_ents_long.csv')
test = test[['test_id','similarity_score']]
test_merged = test.merge(nn_out_test)
nn_out_test.head()

,nn_out,test_id,question1,question2,last_char,avg_shared_words,word_count_diff,levenshtein,shared_words_pcnt,avg_shared_trigrams,shared_bigram_pcnt,shared_trigram_pcnt,avg_shared_quadgrams,shared_quadgram_pcnt,tfidf_word_match_share
0,0.998650,15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...,1,4,2,0.505376,0.250000,1,0.133333,0.071429,0,0.0,0.35100
1,0.637905,20,Why do rockets look white?,Why are rockets and boosters painted white?,1,3,2,0.637681,0.500000,0,0.000000,0.000000,0,0.0,0.40878
2,0.075683,21,What's causing someone to be jealous?,What can I do to avoid being jealous of someone?,1,1,4,0.447059,0.125000,0,0.000000,0.000000,0,0.0,0.00000
3,0.002341,23,How much is 30 kV in HP?,Where can I find a conversion chart for CC to ...,1,0,4,0.074074,0.000000,0,0.000000,0.000000,0,0.0,0.00000
4,0.099990,34,What is the best travel website in spain?,What is the best travel website?,1,5,2,0.876712,0.714286,3,0.666667,0.600000,2,0.5,0.73111


In [34]:
x_test = test_merged[feature_list]
x_test = x_test.fillna(0)

In [35]:
logreg = LogisticRegression()
logreg.fit(x, y)
y_pred = logreg.predict(x_test)

In [36]:
predict = pd.DataFrame({"test_id":test_merged['test_id'], "is_duplicate":y_pred})
predict.to_csv("logregattempt6.csv", index=False)

In [37]:
predict.head()

,is_duplicate,test_id
0,1,15
1,1,20
2,0,21
3,0,23
4,0,34
